In [1]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Log\data_loader.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_MP\Data\Feature_Selection\Filtered_Features.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           4209 non-null   int64  
 1   id                   4209 non-null   float64
 2   ref                  4209 non-null   float64
 3   subtitle             4209 non-null   float64
 4   creatorname          4209 non-null   float64
 5   totalbytes           4209 non-null   float64
 6   lastupdated          4209 non-null   float64
 7   downloadcount        4209 non-null   float64
 8   title                4209 non-null   float64
 9   tags                 4209 non-null   float64
 10  cluster              4209 non-null   int64  
 11  anomaly              4209 non-null   float64
 12  download_view_ratio  4209 non-null   float64
 13  num_tags             4209 non-null   float64
 14  title_length         4209 non-null   float64
 15  subtitle_length      4209 non-null   f

In [30]:
df.head()

,Unnamed: 0,id,ref,subtitle,creatorname,totalbytes,lastupdated,downloadcount,title,tags,...,download_view_ratio,num_tags,title_length,subtitle_length,creator_popularity,owner_popularity,lastupdated_month,lastupdated_day,lastupdated_weekday,owner_popularity_cat
0,0,1.652531,-1.497901,-1.343173,-1.482264,-0.205871,1.440289,-0.334293,-1.148158,-1.305056,...,0.054731,0.644314,0.521569,1.310725,0.0,-0.06614,0.890462,0.152226,0.535769,0
1,1,1.608563,-1.581849,0.951414,-1.578557,-0.205876,1.295437,0.105377,-0.349943,0.271367,...,0.015678,0.321751,1.770817,0.682214,0.0,-0.06614,0.890462,-1.626624,-0.467145,0
2,2,1.672062,-1.224657,-0.147321,-1.218896,-0.205877,1.522592,-0.276472,0.955542,1.099529,...,0.080659,0.063700,0.789265,0.853626,0.0,-0.06614,0.890462,1.152829,1.538684,0
3,3,1.690704,-1.434528,-1.140709,-1.420537,-0.205814,1.599133,-0.329493,-1.527142,-0.562436,...,0.141278,0.708827,0.253873,0.796489,0.0,-0.06614,1.189175,-1.515446,1.538684,0
4,4,1.583257,-1.599133,-1.353049,-1.588433,-0.205741,1.204905,0.203582,-1.403888,-0.790862,...,0.161541,0.902365,0.878497,1.310725,0.0,-0.06614,0.591749,0.485760,1.037227,0


In [4]:
df["cluster"].unique()

array([0, 1, 2, 4, 3])

In [6]:
unique_counts = df.nunique().reset_index()
unique_counts.columns = ['Column', 'Unique_Values']
print(unique_counts)

                 Column  Unique_Values
0            Unnamed: 0           4209
1                    id           4209
2                   ref           4209
3              subtitle           4209
4           creatorname           4209
5            totalbytes           4134
6           lastupdated           4209
7         downloadcount           3495
8                 title           4139
9                  tags           3615
10              cluster              5
11              anomaly              2
12  download_view_ratio           4137
13             num_tags             73
14         title_length             45
15      subtitle_length             65
16   creator_popularity              1
17     owner_popularity              4
18    lastupdated_month             12
19      lastupdated_day             31
20  lastupdated_weekday              7


In [29]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib


le_owner = LabelEncoder()
df["owner_popularity_cat"] = le_owner.fit_transform(df["owner_popularity"])


target_cols = ["cluster", "owner_popularity_cat"]

X = df.drop(columns=target_cols + ["Video ID"], errors="ignore")  # Video ID modelga kerak emas
y = df[target_cols]


X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y["cluster"]  # stratify faqat 1 target bo‘yicha mumkin
)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


base_model = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)
multi_model = MultiOutputClassifier(base_model)


multi_model.fit(X_train_scaled, y_train)


y_pred = multi_model.predict(X_test_scaled)


print("Accuracy (cluster):", accuracy_score(y_test["cluster"], y_pred[:,0]))
print("Accuracy (owner_popularity_cat):", accuracy_score(y_test["owner_popularity_cat"], y_pred[:,1]))

print("\nClassification Report (cluster):")
print(classification_report(y_test["cluster"], y_pred[:,0]))

print("\nClassification Report (owner_popularity_cat):")
print(classification_report(y_test["owner_popularity_cat"], y_pred[:,1]))


Accuracy (cluster): 0.9881235154394299
Accuracy (owner_popularity_cat): 0.998812351543943

Classification Report (cluster):
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       774
           1       0.92      0.85      0.88        40
           2       1.00      0.71      0.83         7
           3       1.00      0.50      0.67         2
           4       0.95      0.95      0.95        19

    accuracy                           0.99       842
   macro avg       0.97      0.80      0.87       842
weighted avg       0.99      0.99      0.99       842


Classification Report (owner_popularity_cat):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       838
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         1
           3       1.00      1.00      1.00         3

    accuracy                           1.00       842
   macro avg  

c:\Users\Rasulbek907\Desktop\Project_MP\mpvenv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Rasulbek907\Desktop\Project_MP\mpvenv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Rasulbek907\Desktop\Project_MP\mpvenv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

In [34]:
import joblib
import os

# Model saqlanadigan folder
model_dir = r"C:\Users\Rasulbek907\Desktop\Project_MP\Models\Pipeline_Models"
os.makedirs(model_dir, exist_ok=True)  # Agar folder mavjud bo'lmasa, yaratadi

# Model fayl nomi
model_path = os.path.join(model_dir, "multioutput_rf_model.joblib")

# Modelni saqlash
joblib.dump(multi_model, model_path)

print(f"Model saqlandi: {model_path}")


Model saqlandi: C:\Users\Rasulbek907\Desktop\Project_MP\Models\Pipeline_Models\multioutput_rf_model.joblib


# Multi-output classification  uchun bu user_input ma’lumotlari

In [31]:
user_input = [
    {
        "Unnamed: 0": 0,
        "id": 1.652531,
        "ref": -1.497901,
        "subtitle": -1.343173,
        "creatorname": -1.482264,
        "totalbytes": -0.205871,
        "lastupdated": 1.440289,
        "downloadcount": -0.334293,
        "title": -1.148158,
        "tags": -1.305056,
        "cluster": 0,
        "anomaly": 0.054731,
        "download_view_ratio": 0.644314,
        "num_tags": 0.521569,
        "title_length": 1.310725,
        "subtitle_length": 0.0,
        "creator_popularity": -0.06614,
        "owner_popularity": 0.890462,
        "lastupdated_month": 0.152226,
        "lastupdated_day": 0.535769,
        "lastupdated_weekday": 0,
        "owner_popularity_cat": 0
    },
    {
        "Unnamed: 0": 1,
        "id": 1.608563,
        "ref": -1.581849,
        "subtitle": 0.951414,
        "creatorname": -1.578557,
        "totalbytes": -0.205876,
        "lastupdated": 1.295437,
        "downloadcount": 0.105377,
        "title": -0.349943,
        "tags": 0.271367,
        "cluster": 1,
        "anomaly": 0.015678,
        "download_view_ratio": 0.321751,
        "num_tags": 1.770817,
        "title_length": 0.682214,
        "subtitle_length": 0.0,
        "creator_popularity": -0.06614,
        "owner_popularity": 0.890462,
        "lastupdated_month": -1.626624,
        "lastupdated_day": -0.467145,
        "lastupdated_weekday": 0,
        "owner_popularity_cat": 0
    },
    {
        "Unnamed: 0": 2,
        "id": 1.672062,
        "ref": -1.224657,
        "subtitle": -0.147321,
        "creatorname": -1.218896,
        "totalbytes": -0.205877,
        "lastupdated": 1.522592,
        "downloadcount": -0.276472,
        "title": 0.955542,
        "tags": 1.099529,
        "cluster": 2,
        "anomaly": 0.080659,
        "download_view_ratio": 0.063700,
        "num_tags": 0.789265,
        "title_length": 0.853626,
        "subtitle_length": 0.0,
        "creator_popularity": -0.06614,
        "owner_popularity": 0.890462,
        "lastupdated_month": 1.152829,
        "lastupdated_day": 1.538684,
        "lastupdated_weekday": 0,
        "owner_popularity_cat": 0
    }
]

df_input = pd.DataFrame(user_input)
print(df_input.head())


   Unnamed: 0        id       ref  subtitle  creatorname  totalbytes  \
0           0  1.652531 -1.497901 -1.343173    -1.482264   -0.205871   
1           1  1.608563 -1.581849  0.951414    -1.578557   -0.205876   
2           2  1.672062 -1.224657 -0.147321    -1.218896   -0.205877   

   lastupdated  downloadcount     title      tags  ...  download_view_ratio  \
0     1.440289      -0.334293 -1.148158 -1.305056  ...             0.644314   
1     1.295437       0.105377 -0.349943  0.271367  ...             0.321751   
2     1.522592      -0.276472  0.955542  1.099529  ...             0.063700   

   num_tags  title_length  subtitle_length  creator_popularity  \
0  0.521569      1.310725              0.0            -0.06614   
1  1.770817      0.682214              0.0            -0.06614   
2  0.789265      0.853626              0.0            -0.06614   

   owner_popularity  lastupdated_month  lastupdated_day  lastupdated_weekday  \
0          0.890462           0.152226         0.

In [32]:
required_cols = [
    'Unnamed: 0',
    'id',
    'ref',
    'subtitle',
    'creatorname',
    'totalbytes',
    'lastupdated',
    'downloadcount',
    'title',
    'tags',
    'cluster',
    'anomaly',
    'download_view_ratio',
    'num_tags',
    'title_length',
    'subtitle_length',
    'creator_popularity',
    'owner_popularity',
    'lastupdated_month',
    'lastupdated_day',
    'lastupdated_weekday'
]


In [33]:
for col in required_cols:
    if col not in df_input.columns:
        # Categorical ustunlar uchun 'Unknown' qiymat beramiz
        if col in ['creatorname', 'd', 'title', 'lastupdated_day']:  
            df_input[col] = 'Unknown'
        else:
            # Qolgan numeric ustunlar uchun 0 bilan to‘ldiramiz
            df_input[col] = 0

In [35]:
import joblib

# Model fayl yo‘li
model_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Models\Pipeline_Models\multioutput_rf_model.joblib"

# Modelni yuklash
loaded_model = joblib.load(model_path)

print("Model muvaffaqiyatli yuklandi!")

Model muvaffaqiyatli yuklandi!


In [36]:
print("Pipeline ustunlari:", preprocessor.feature_names_in_)
print("df_input ustunlari:", df_input.columns.tolist())

Pipeline ustunlari: ['ref' 'subtitle' 'creatorname' 'totalbytes' 'lastupdated' 'downloadcount'
 'title' 'tags' 'anomaly' 'download_view_ratio' 'num_tags' 'title_length'
 'subtitle_length' 'creator_popularity' 'lastupdated_month'
 'lastupdated_day' 'lastupdated_weekday']
df_input ustunlari: ['Unnamed: 0', 'id', 'ref', 'subtitle', 'creatorname', 'totalbytes', 'lastupdated', 'downloadcount', 'title', 'tags', 'cluster', 'anomaly', 'download_view_ratio', 'num_tags', 'title_length', 'subtitle_length', 'creator_popularity', 'owner_popularity', 'lastupdated_month', 'lastupdated_day', 'lastupdated_weekday', 'owner_popularity_cat']


In [61]:
import pandas as pd
import joblib
import numpy as np

# 🔹 Modelni yuklash
model_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Models\Pipeline_Models\multioutput_rf_model.joblib"
loaded_model = joblib.load(model_path)
print("✅ Model muvaffaqiyatli yuklandi!")

# 🔹 df_input ustunlarini aniqlash
df_input_cols = df_input.columns.tolist()

# 🔹 Model fit qilgan ustunlar soni (base estimator)
n_features_expected = loaded_model.estimators_[0].n_features_in_
print(f"Model {n_features_expected} ustun bilan fit qilingan.")

# 🔹 df_input dan ustunlarni tanlash / yetishmaydigan ustunlar uchun 0 bilan to‘ldirish
X_features = pd.DataFrame()
for i, col in enumerate(df_input_cols):
    if i >= n_features_expected:
        break
    X_features[col] = df_input[col]

# 🔹 Agar hali yetishmayotgan ustunlar bo‘lsa, 0 bilan to‘ldirish
while X_features.shape[1] < n_features_expected:
    missing_col = f"missing_feature_{X_features.shape[1]+1}"
    X_features[missing_col] = 0

print(f"X_features tayyor: {X_features.shape[1]} ustun")

# 🔹 Predict qilish
predictions = loaded_model.predict(X_features)

# 🔹 Natijalarni saqlash
df_results = df_input.copy()
df_results['Predicted Cluster'] = predictions[:,0]
df_results['Predicted Anomaly'] = predictions[:,1]

print(df_results[['cluster','anomaly','Predicted Cluster','Predicted Anomaly']].head())


✅ Model muvaffaqiyatli yuklandi!
Model 20 ustun bilan fit qilingan.
X_features tayyor: 20 ustun
   cluster   anomaly  Predicted Cluster  Predicted Anomaly
0        0  0.054731                  0                  0
1        1  0.015678                  0                  0
2        2  0.080659                  0                  0


c:\Users\Rasulbek907\Desktop\Project_MP\mpvenv\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Users\Rasulbek907\Desktop\Project_MP\mpvenv\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [65]:
df_results['Predicted Cluster'] = df_results['Predicted Cluster'].astype(int)
df_results['Predicted Anomaly'] = df_results['Predicted Anomaly'].astype(int)

In [67]:
from sklearn.metrics import accuracy_score, classification_report

# 🔹 Predicted va original targetni intga o‘zgartirish
df_results['Predicted Anomaly'] = df_results['Predicted Anomaly'].astype(int)
df_input['anomaly'] = df_input['anomaly'].astype(int)

df_results['Predicted Cluster'] = df_results['Predicted Cluster'].astype(int)
df_input['cluster'] = df_input['cluster'].astype(int)

# 🔹 Cluster uchun metrics
accuracy_cluster = accuracy_score(df_input['cluster'], df_results['Predicted Cluster'])
print(f"Accuracy for Cluster: {accuracy_cluster:.4f}")
print("\nClassification Report for Cluster:")
print(classification_report(df_input['cluster'], df_results['Predicted Cluster']))

# 🔹 Anomaly uchun metrics
accuracy_anomaly = accuracy_score(df_input['anomaly'], df_results['Predicted Anomaly'])
print(f"\nAccuracy for Anomaly: {accuracy_anomaly:.4f}")
print("\nClassification Report for Anomaly:")
print(classification_report(df_input['anomaly'], df_results['Predicted Anomaly']))


Accuracy for Cluster: 0.3333

Classification Report for Cluster:
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         1
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1

    accuracy                           0.33         3
   macro avg       0.11      0.33      0.17         3
weighted avg       0.11      0.33      0.17         3


Accuracy for Anomaly: 1.0000

Classification Report for Anomaly:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3



c:\Users\Rasulbek907\Desktop\Project_MP\mpvenv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Rasulbek907\Desktop\Project_MP\mpvenv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Rasulbek907\Desktop\Project_MP\mpvenv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri